# 01 - Single Inheritance

Python supports multiple inheritance, unlike languages like Java which only support single inheritance (but allows multiple interfaces), but we'll look at that later.

#### Some basics

If our classes tend to form a natural hierarchy, then creating an inheritance tree may be useful.

Classes lower down in the hierarchy will **inherit** characteristics (state and behaviour i.e properties and methods) from those higher up. 

But they can also **extend** to have characteristics that those higher up didn't have.

They can also **override** characteristics in those higher up.

**Inherits from / subclasses / is a child of / derives from** are all synonymous terms. Note that the terms **child** and **parent** refer to direct relationships while **ancestor** refers to indirect relationships. So if **A** inherits from **B** which inherits from **C**, then **C** is *not* a parent of **A** but it *is* an ancestor.

#### `isinstance` vs `type` vs `issubclass`

If `Student` inherits from `Person`, then any instances of `Student` are automatically instances of `Person`. But, `Person` instances are of course not `Student` instances.

`isinstance` does not look at direct relationships; since all objects inherit from `object`, `isinstance(<anything>, object)` will always be `True`.

The same applies for `issubclass` but **issubclass** can only be used to inspect inheritance relationships between **classes** not instances.

`type` returns the class that was used to create the instance - it does not look at inheritance.

In [2]:
class Person:
    pass

class Student(Person):
    pass

s1 = Student()

isinstance(s1, Person)

True

In [3]:
class CollegeStudent(Student):
    pass

issubclass(CollegeStudent, Person)

True

One useful thing to remember is that the types printed out using `type` aren't necessarily the builtin objects themselves but rather a string representation. But the actual types can be found in other modules. Here are two examples below:

In [16]:
import types

def my_func():
    pass

types.FunctionType is type(my_func)

True

In [17]:
import math

types.ModuleType is type(math)

True

# 02 - The object Class

Despite being lowercase, `object` is a class, not an instance of some other class.

If one of our classes does not override a characteristic from `object` when it inherited it, then that characteristic will be **identical** to the one found in our class. For example, if we do not implement `__init__` method, then the `__init__` method of `object` will be called.

In [18]:
class Person:
    pass

Person.__init__ is object.__init__

True

But instances will technically have a different `__init__`.

In [26]:
p1 = Person()

p1.__init__, Person.__init__, object.__init__

(<method-wrapper '__init__' of Person object at 0x0000019FDF8A12A0>,
 <slot wrapper '__init__' of 'object' objects>,
 <slot wrapper '__init__' of 'object' objects>)

# 03 - Overriding

# 04 - Extending

# 05 - Delegating to Parent

# 06 - Slots

# 07 - Slots and Single Inheritance